In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

embedding = OllamaEmbeddings(model="qwen2.5")
llm = ChatOllama(
    model="qwen2.5",
    temperature=0.0,
    max_tokens=100,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.0,
)

In [6]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

embedding = OllamaEmbeddings(model="qwen2.5")
llm = ChatOllama(model="qwen2.5", temperature=0.0, max_tokens=100, top_p=0.9, top_k=50, repetition_penalty=1.0)


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [2]:
from langchain.vectorstores import Chroma

persist_directory = "../docs/chroma/matplotlib/"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

print(vectordb._collection.count())

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_62201/329282471.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


27


In [ ]:
question = "这节课的主要话题是什么"
docs = vectordb.similarity_search(question, k=3)
len(docs)
pretty_print_docs(docs)

3

In [9]:
# 导入检索式问答链
from langchain.chains import RetrievalQA

# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

# 可以以该方式进行检索问答
question = "这节课的主要话题是什么"
result = qa_chain({"query": question})

print(result["result"])

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_62201/2518845439.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


这节课的主要话题是关于Matplotlib的使用，包括如何绘制图表、设置坐标轴类型（如对数坐标）、创建子图以及数据可视化的一些基本概念。具体来说，内容涉及以下几个方面：

1. 使用`plt.subplots()`创建图形和子图。
2. 设置坐标轴的尺度，例如使用`set_xscale('log')`来设置对数坐标。
3. 如何加载并绘制数据集中的信息（以一个假设的数据集为例）。
4. 对比面向对象（OO）模式与Pyplot模式两种绘图方式，并提供了一个通用的绘图模板。

此外，还介绍了Matplotlib中的一些基本概念，如Figure、Axes等容器及其属性和方法。这些内容为理解和使用Matplotlib进行数据可视化打下了基础。


In [24]:
# 中文版
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [25]:
question = "这门课会学习 Python 吗"
result = qa_chain({"query": question})
print(result["result"])

这门课程涉及使用Python和Matplotlib进行数据可视化，但主要问题是询问是否会学习Python本身。根据提供的内容，可以推断这门课程会涉及到Python编程，因为提到了使用NumPy生成数据以及使用Matplotlib绘制图表。但是具体是否系统性地教授Python语言的基础知识，则需要更多信息来确定。

感谢您的提问！


In [16]:
print(result["source_documents"][0])

page_content='fig, ax = plt.subplots(figsize=(4,3))
ax.grid(True)
使⽤  set_xscale 可以设置坐标轴的规度（指对数坐标等）
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
for j in range(2):
    axs[j].plot(list('abcd'), [10**i for i in range(4)])
    if j==0:
        axs[j].set_yscale('log')
    else:
        pass
fig.tight_layout()
思考题
墨尔本 1981 年⾄ 1990 年的每⽉温度情况
数据集来⾃ github 仓库下 data/layout_ex1.csv
请利⽤数据，画出如下的图：
画出数据的散点图和边际分布
⽤  np.random.randn(2, 150) ⽣成⼀组⼆维数据，使⽤两种⾮均匀⼦图的分割⽅法，做出该数据对应的散点图和边际分布图' metadata={'creationdate': '2023-07-11T13:58:18+00:00', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'moddate': '2023-07-12T10:54:35+08:00', 'page': 3, 'page_label': '4', 'producer': 'Skia/PDF m114', 'source': '../docs/matplotlib/第三回：布局格式定方圆.pdf', 'total_pages': 5}


In [18]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(), chain_type="map_reduce"
)

question = "这门课会学习 Python 吗"
result = qa_chain_mr({"query": question})

print(result["result"])

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


根据给定的文档片段，无法确定这门课是否会学习 Python。文档中未直接提到课程内容是否包括 Python。


In [19]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm, retriever=vectordb.as_retriever(), chain_type="refine"
)
question = "这门课会学习 Python 吗"
result = qa_chain_mr({"query": question})
print(result["result"])

好的，根据提供的新上下文，我们可以进一步细化和补充之前的答案。以下是结合新信息后的详细回答：

### 课程内容概览

1. **数据可视化基础**
   - 使用`matplotlib`绘制基本图表（如线性、二次和三次函数）。
   
2. **通用绘图模板**
   - 准备数据
   - 设置绘图样式
   - 定义布局
   - 绘制图像
   - 添加标签、文字和图例

### Python 和 matplotlib 的学习

课程会涉及Python编程，特别是使用`matplotlib`进行数据可视化。具体步骤如下：

1. **准备数据**
   ```python
   import numpy as np
   x = np.linspace(0, 2, 100)
   y_linear = x
   y_quadratic = x**2
   y_cubic = x**3
   ```

2. **设置绘图样式（可选）**
   ```python
   import matplotlib.pyplot as plt
   mpl.rc('lines', linewidth=4, linestyle='-.')
   ```

3. **定义布局**
   - 使用`Figure`容器创建图表背景。
   - 使用`Axes`容器绘制具体图像。

4. **绘制图像**
   ```python
   fig = plt.figure()
   ax1 = fig.add_subplot(211)  # 添加一个子图到Figure中
   for ax in fig.axes:
       ax.grid(True)
   
   ax1.plot(x, y_linear, label='Linear')
   ax1.plot(x, y_quadratic, label='Quadratic')
   ax1.plot(x, y_cubic, label='Cubic')
   ```

5. **添加标签、文字和图例**
   ```python
   ax1.set_xlabel('X-axis')
   ax1.set_ylabel('Y-axis')
   ax1.set_title('Plot of Linear, Quadrati

In [21]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

question = "这门课会学习 Python 吗？"
result = qa_chain({"query": question})
print(result["result"])

这门课程主要介绍使用 Python 的 matplotlib 库进行数据可视化，但并没有直接提到是否教授 Python 语言的基础知识。因此，根据提供的上下文信息，我们无法确定这门课是否会学习 Python。不过通常情况下，学习使用 matplotlib 进行数据可视化的课程都会涵盖一定的 Python 基础知识。如果你对 Python 编程感兴趣或不熟悉，可能需要在开始之前先自学一些基础知识。


In [23]:
question = "为什么需要这一前提？"
result = qa_chain({"query": question})
result["result"]

'您提到的前提不太清晰，没有明确指出是哪个前提。如果您是指上述代码或内容中的某个特定部分，请提供更多的上下文信息，这样我才能准确回答您的问题。如果是在询问关于数据可视化中使用极坐标系、非均匀子图分割或者通用绘图模板等具体技术的必要性，那么这些技术的应用通常是为了更好地展示和理解数据。\n\n例如：\n- 使用极坐标系可以帮助更直观地表示周期性或方向性的数据。\n- 非均匀子图分割可以灵活地调整图表布局以适应不同的数据展示需求。\n- 通用绘图模板提供了一种结构化的方式来组织绘图过程，有助于初学者理解和扩展复杂的图表绘制。\n\n请具体说明您所指的前提是什么，以便我能给出更准确的回答。'